# Парсинг, практика

*Анастасия Паршина, НИУ ВШЭ*

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep

## Задание 1

Дана ссылка на сайт [IMDb](https://www.imdb.com/calendar/?ref_=nv_mv_cal). Сохраните название и ссылки всех фильмов на странице. 

In [2]:
url = 'https://www.imdb.com/calendar/?ref_=nv_mv_cal'
print(url)

https://www.imdb.com/calendar/?ref_=nv_mv_cal


In [3]:
page = requests.get(url)
page.status_code # 200 - все хорошо

200

In [4]:
soup = BeautifulSoup(page.text)
#soup

In [5]:
dates = soup.find_all('h4') # все наши даты
dates

[<h4>19 October 2021</h4>,
 <h4>22 October 2021</h4>,
 <h4>29 October 2021</h4>,
 <h4>03 November 2021</h4>,
 <h4>05 November 2021</h4>,
 <h4>10 November 2021</h4>,
 <h4>12 November 2021</h4>,
 <h4>19 November 2021</h4>,
 <h4>24 November 2021</h4>,
 <h4>26 November 2021</h4>,
 <h4>03 December 2021</h4>,
 <h4>10 December 2021</h4>,
 <h4>17 December 2021</h4>,
 <h4>22 December 2021</h4>,
 <h4>25 December 2021</h4>,
 <h4>07 January 2022</h4>,
 <h4>14 January 2022</h4>,
 <h4>21 January 2022</h4>,
 <h4>28 January 2022</h4>,
 <h4>04 February 2022</h4>,
 <h4>11 February 2022</h4>,
 <h4>18 February 2022</h4>,
 <h4>04 March 2022</h4>,
 <h4>11 March 2022</h4>,
 <h4>18 March 2022</h4>,
 <h4>25 March 2022</h4>,
 <h4>08 April 2022</h4>,
 <h4>15 April 2022</h4>,
 <h4>22 April 2022</h4>,
 <h4>06 May 2022</h4>,
 <h4>20 May 2022</h4>,
 <h4>22 May 2022</h4>,
 <h4>27 May 2022</h4>,
 <h4>03 June 2022</h4>,
 <h4>10 June 2022</h4>,
 <h4>17 June 2022</h4>,
 <h4>24 June 2022</h4>,
 <h4>01 July 2022</h4>,
 <h4

In [6]:
soup.find_all('h4')[0].text

'19 October 2021'

In [7]:
# достать ссылки можно так, но как понять, к какой дате они относятся??
soup.find_all('a', attrs = {'href' : re.compile('title/tt')}) 

[<a href="/title/tt12335692/?ref_=rlm">Last Man Down</a>,
 <a href="/title/tt1160419/?ref_=rlm">Dune</a>,
 <a href="/title/tt8847712/?ref_=rlm">The French Dispatch</a>,
 <a href="/title/tt7504818/?ref_=rlm">Ron's Gone Wrong</a>,
 <a href="/title/tt8956324/?ref_=rlm">Warning</a>,
 <a href="/title/tt9639470/?ref_=rlm">Last Night in Soho</a>,
 <a href="/title/tt7740510/?ref_=rlm">Antlers</a>,
 <a href="/title/tt13544716/?ref_=rlm">My Hero Academia: World Heroes' Mission</a>,
 <a href="/title/tt10925852/?ref_=rlm">A Mouthful of Air</a>,
 <a href="/title/tt6992978/?ref_=rlm">The Souvenir: Part II</a>,
 <a href="/title/tt9274670/?ref_=rlm">13 Minutes</a>,
 <a href="/title/tt8400856/?ref_=rlm">The Grand Duke of Corsica</a>,
 <a href="/title/tt9032400/?ref_=rlm">Eternals</a>,
 <a href="/title/tt12536294/?ref_=rlm">Spencer</a>,
 <a href="/title/tt11388416/?ref_=rlm">Ida Red</a>,
 <a href="/title/tt6852672/?ref_=rlm">Violet</a>,
 <a href="/title/tt11738830/?ref_=rlm">The Beta Test</a>,
 <a href=

In [8]:
# пробуем по-другому
info = soup.find_all('div', {'id': 'main'})[0].find_all('ul')
#info

Теперь количество дат соответствует количеству элементов полученного выше списка.

In [9]:
info[0]

<ul>
<li>
<a href="/title/tt12335692/?ref_=rlm">Last Man Down</a> (2021)
                        </li>
</ul>

In [10]:
info[0].find_all('a')[0]

<a href="/title/tt12335692/?ref_=rlm">Last Man Down</a>

In [11]:
'https://www.imdb.com' + info[0].find_all('a')[0].get('href')

'https://www.imdb.com/title/tt12335692/?ref_=rlm'

In [12]:
urls = ['https://www.imdb.com' + i.get('href') for i in info[0].find_all('a')]
urls # это ссылки на все фильмы соответствующей даты

['https://www.imdb.com/title/tt12335692/?ref_=rlm']

In [13]:
info[0].find_all('a')[0].text

'Last Man Down'

In [14]:
names = [i.text for i in info[0].find_all('a')]
names

['Last Man Down']

In [15]:
movies = {}
for d in range(len(dates)):
    
    names = [i.text for i in info[d].find_all('a')]
    urls = ['https://www.imdb.com' + i.get('href') for i in info[d].find_all('a')]
    
    urls_names = {}
    for x in range(len(names)):
        urls_names[names[x]] = [urls[x]] # ключом словаря будет дата, а значением — ссылка
    
    movies[dates[d].text] = urls_names
#movies

In [18]:
rows = ['Ссылка']
df = pd.DataFrame(movies['05 November 2021'], index = rows)
df

,Eternals,Spencer,Ida Red,Violet,The Beta Test,Dangerous
Ссылка,https://www.imdb.com/title/tt9032400/?ref_=rlm,https://www.imdb.com/title/tt12536294/?ref_=rlm,https://www.imdb.com/title/tt11388416/?ref_=rlm,https://www.imdb.com/title/tt6852672/?ref_=rlm,https://www.imdb.com/title/tt11738830/?ref_=rlm,https://www.imdb.com/title/tt3876910/?ref_=rlm


In [19]:
df.columns = [['05 November 2021']*len(movies['05 November 2021'].keys()),
                            list(movies['05 November 2021'].keys())]

## Задание 2

Соберите информацию о сотрудниках [кафедры высшей математики](https://hmat.hse.ru/persons). Если сотрудник ведет какие-то курсы, то также название курса и ссылка на его страницу.

In [23]:
url = 'https://hmat.hse.ru/persons'
page = requests.get(url)
page.status_code

200

In [24]:
soup = BeautifulSoup(page.text)
#soup

In [25]:
info = soup.find_all('div', {'class': 'fa-person__box'})
#info

In [26]:
info[14]

<div class="fa-person__box">
<a class="fa-person__name" href="//www.hse.ru/org/persons/207912918">
							Латышев Александр Михайлович
						</a>
<p class="fa-person__info">Старший преподаватель</p>
</div>

In [27]:
name = info[14].find('a').text.strip()
name

'Латышев Александр Михайлович'

In [28]:
degree = info[14].find('p').text.strip()
degree

'Старший преподаватель'

In [29]:
url_person = 'https:' + info[14].find('a').get('href')
url_person

'https://www.hse.ru/org/persons/207912918'

In [30]:
url0 = 'https://www.hse.ru/org/persons/207912918'
url0

'https://www.hse.ru/org/persons/207912918'

In [31]:
page0 = requests.get(url0)
page0.status_code

200

In [32]:
soup0 = BeautifulSoup(page0.text)
#soup0

In [33]:
courses = soup0.find_all('div', {'tab-node':'edu-courses'})
courses

[<div class="b-person-data" tab-index="teaching" tab-node="edu-courses"><a id="m-teaching"></a><h3>Учебные курсы (2021/2022 уч. год)</h3><ul class="g-ul g-list g-list_closer"><li><a class="link" href="https://www.hse.ru/edu/courses/395858382">Математический анализ</a> (Бакалавриат; где читается: <a class="link" href="https://chemistry.hse.ru/">Факультет химии</a>; 2-й курс, 1-4 модуль)<span class="language-label">Рус</span></li><li><span class="link edu-courses-archive-toogle">Архив учебных курсов</span></li></ul></div>]

In [34]:
courses_info = courses[0].find_all('a', {'class':'link'})
courses_info

[<a class="link" href="https://www.hse.ru/edu/courses/395858382">Математический анализ</a>,
 <a class="link" href="https://chemistry.hse.ru/">Факультет химии</a>]

In [38]:
staff = {}

for i in info:
    name = i.find('a').text.strip()
    degree = i.find('p').text.strip()
    url_person = 'https:' + i.find('a').get('href')
    
    page0 = requests.get(url_person)
    soup0 = BeautifulSoup(page0.text)
    
    sleep(1)
    
    courses = soup0.find_all('div', {'tab-node':'edu-courses'})
    if len(courses) != 0:
        courses_info = courses[0].find_all('a', {'class':'link'})

        empty_dict = {}

        for i in range(0, len(courses_info), 2):

            key = courses_info[i].get('href')
            value_name = courses_info[i].text
            value_fac = courses_info[i+1].text

            empty_dict[key] = [value_name, value_fac]

        staff[f'{name}({degree})'] = empty_dict
    

KeyboardInterrupt: 

In [ ]:
rows = ['Название', 'Факультет']
df = pd.DataFrame(staff[list(staff.keys())[0]], index = rows)

df.columns = [[list(staff.keys())[0]]*len(staff[list(staff.keys())[0]].keys()),
                            list(staff[list(staff.keys())[0]].keys())]

df

In [ ]:
for i in list(staff.keys())[1:]:
    df_to_join = pd.DataFrame(staff[i], index = rows)
    df_to_join.columns = [[i]*len(staff[i].keys()),
                            list(staff[i].keys())]
    
    df = df.join(df_to_join)

In [ ]:
df.transpose()

## Задание 3

Дана ссылка на первую страницу [каталога книжных новинок](https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=). Соберите информацию о книгах с первых трех страниц. 

In [39]:
urls = []

for i in range(1,4):
    urls.append(f'https://www.bgshop.ru/catalog/group?id=444&page={i}&sort=1&instock=')
urls

['https://www.bgshop.ru/catalog/group?id=444&page=1&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=2&sort=1&instock=',
 'https://www.bgshop.ru/catalog/group?id=444&page=3&sort=1&instock=']

In [40]:
page = requests.get(urls[0])
page.status_code

200

In [41]:
soup = BeautifulSoup(page.text)
#soup

In [42]:
product = soup.find_all('div', {'class':'product'})[0]
product

<div class="product">
<a class="img_link" href="/Catalog/GetFullDescription?id=10799787&amp;type=1"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10799787.jpg&amp;width=200"/></a>
<div class="text">
<div class="author">Г. Тайс</div>
<h3><a href="/Catalog/GetFullDescription?id=10799787&amp;type=1" id="p_title_10799787">Вся твоя ложь</a></h3>
<p class="price">
<span class="im_price_title" style="font-size:0.85em;">Цена в интернет-магазине: </span><span id="p_price_10799787" style="display:block; font-weight:bold; font-size:1.4em; margin:6px 0 5px; color:#111;">
                                589,00 <span style="font-size:.8em; font-weight:normal;">руб.</span>
</span>
<span style="display:block;"> </span>
</p>
<p class="status im_status_title">
в наличии                        </p>
<p class="buttons">
<span class="btn btn-primary" id="add_10799787" onclick="AddToBasket(10799787)"><i class="fa fa-shopping-cart"></i> В корзину</span>
<a class="btn 

In [43]:
# ссылка
url0 = 'https://www.bgshop.ru/' + product.find_all('a')[0].get('href')
url0

'https://www.bgshop.ru//Catalog/GetFullDescription?id=10799787&type=1'

In [44]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)
#soup0

In [45]:
main_info = soup0.find_all('div', {'id':'productMain'})[0]
main_info

<div class="row box" id="productMain">
<div class="col-md-4">
<a data-fancybox="gallery" href="/imagehandler.ashx?filename=10799787.jpg&amp;width=400"><img alt="Вся твоя ложь" class="product_photo zoomIn" src="/imagehandler.ashx?filename=10799787.jpg&amp;width=200" title="Вся твоя ложь" width="200"/></a>
</div>
<div class="col-md-7">
<div class="item-desc">
<p class="goToDescription"><a class="scroll-to" href="/Catalog/Search?query=Тайс Г.&amp;page=1">
Тайс Г.             </a></p>
<h1 id="p_title_10799787">Вся твоя ложь</h1>
<p class="item-status">в наличии</p>
<div id="price">Цена в интернет-магазине: <span id="p_price_10799787" style="color: #a42921; font-size:1.2rem;">729,00 руб.</span></div>
<p class="buttons">
<span class="btn btn-primary" id="add_10799787" onclick="AddToBasket(10799787)"><i class="fa fa-shopping-cart"></i> В корзину</span>
<a class="btn btn-primary add_but" href="/Basket/Detail" id="added_10799787">Перейти в корзину <i class="fa fa-chevron-right"></i></a>
</p>
<a

In [46]:
author = main_info.find_all('p')[0].text
author

'\r\nТайс Г.             '

In [47]:
title = main_info.find_all('h1')[0].text
title

'Вся твоя ложь'

In [48]:
status = main_info.find_all('p', {'class':'item-status'})[0].text
status

'в наличии'

In [49]:
price = main_info.find_all('div', {'id':'price'})[0].text
price

'Цена в интернет-магазине: 729,00\xa0руб.'

In [50]:
price_clean = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))
price_clean

729.0

In [51]:
books = {}

for i in soup.find_all('div', {'class':'product'}):
    url0 = 'https://www.bgshop.ru/' + i.find_all('a')[0].get('href')
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)
    
    main_info = soup0.find_all('div', {'id':'productMain'})[0]
    author = main_info.find_all('p')[0].text
    title = main_info.find_all('h1')[0].text
    status = main_info.find_all('p', {'class':'item-status'})[0].text
    
    price = main_info.find_all('div', {'id':'price'})[0].text
    price_clean = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))
    
    books[title] = [author, status, price_clean, url0]

А теперь для всех (трех) страниц с книгами:

In [52]:
books = {}
for u in urls:
    page = requests.get(u)
    soup = BeautifulSoup(page.text)
    sleep(1)
    info = soup.find_all('div', {'class':'product'})
    for i in info:
        url0 = 'https://www.bgshop.ru/' + i.find_all('a')[0].get('href')
        page0 = requests.get(url0)
        soup0 = BeautifulSoup(page0.text)
        sleep(1)

        main_info = soup0.find_all('div', {'id':'productMain'})[0]
        author = main_info.find_all('p')[0].text
        title = main_info.find_all('h1')[0].text
        status = main_info.find_all('p', {'class':'item-status'})[0].text

        price = main_info.find_all('div', {'id':'price'})[0].text
        price_change = float(re.findall(r'\d+,\d+', price)[0].replace(',', '.'))

        books[title] = [author, status, price_change, url0]

In [53]:
df = pd.DataFrame(books, index = ['Автор', 'Статус', 'Цена', 'Ссылка']).transpose()
df

,Автор,Статус,Цена,Ссылка
Вся твоя ложь,\r\nТайс Г.,в наличии,729.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Преступления любви,\r\nМаркиз де Сад,в наличии,649.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Кровь и мёд,\r\nШелби Махёрин,,719.0,https://www.bgshop.ru//Catalog/GetFullDescript...
"ВДНХ. Мечта о прекрасном, несбыточном. Живая история выставки",\r\nПанкратов Г. В.,в наличии,1019.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Вселенная Хаяо Миядзаки. Картины великого аниматора в деталях,\r\nГаэль Бертон,в наличии,759.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Сумерки хищников,\r\nМарк Леви,в наличии,509.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Порог,\r\nОлег Рой,в наличии,819.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Двойной контроль,\r\nЭдвард Сент-Обин,в наличии,669.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Рождественский Поросёнок,\r\nРоулинг Дж.К.,в наличии,583.0,https://www.bgshop.ru//Catalog/GetFullDescript...
Вакцинация. Спасение человечества или всеобщая эвтаназия?,\r\nВалентин Катасонов,,1059.0,https://www.bgshop.ru//Catalog/GetFullDescript...


## Задание 4

Дана ссылка на [сайт газеты "Московский комсомолец"](https://www.mk.ru/news/). Соберите ссылки на новости первой страницы, время публикации, заголовок. Пройдите по ссылкам и соберите количество просмотров и хэштеги статьи. 

In [54]:
url = 'https://www.mk.ru/news/'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [55]:
soup.find_all('li', {'class':'news-listing__item'})[0]

<li class="news-listing__item news-listing__item_hot">
<a class="news-listing__item-link" href="https://www.mk.ru/incident/2021/10/16/v-moskve-zaderzhany-kavkazcy-posle-vtorogo-incidenta-v-metro.html">
<span class="news-listing__item-time">09:41</span>
<h3 class="news-listing__item-title">В Москве задержаны кавказцы после второго инцидента в метро</h3>
</a>
</li>

In [56]:
url0 = soup.find_all('li', {'class':'news-listing__item'})[0].find('a').get('href')
url0

'https://www.mk.ru/incident/2021/10/16/v-moskve-zaderzhany-kavkazcy-posle-vtorogo-incidenta-v-metro.html'

In [57]:
time = soup.find_all('li', {'class':'news-listing__item'})[0].find('span').text
time

'09:41'

In [58]:
title = soup.find_all('li', {'class':'news-listing__item'})[0].find('h3').text
title

'В Москве задержаны кавказцы после второго инцидента в метро'

In [59]:
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

In [60]:
soup0.find_all('span', {'class':'meta__item_views'})[0].text.strip()

'1275'

In [61]:
soup0.find('div', {'class':'article__tag'}).text

'\nПолиция\nСКР\nМосква\n'

In [62]:
re.findall(r'[^\n]+', soup0.find('div', {'class':'article__tag'}).text)

['Полиция', 'СКР', 'Москва']

In [64]:
news = {}
for i in soup.find_all('li', {'class':'news-listing__item'}):
    url0 = i.find('a').get('href')
    time = i.find('span').text
    title = i.find('h3').text
    
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    sleep(1)
    
    view = int(soup0.find_all('span', {'class':'meta__item_views'})[0].text.strip())
    
    if soup0.find('div', {'class':'article__tag'}) is None:
        tags = ''
    else:
        tags = ', '.join(re.findall(r'[^\n]+', soup0.find('div', {'class':'article__tag'}).text))
        
    news[title] = [url0, time, view, tags]

ConnectionError: HTTPSConnectionPool(host='www.mk.ru', port=443): Max retries exceeded with url: /social/2021/10/16/uspenskaya-pokazala-pikantnoe-foto-s-baskovym.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ff52736eb50>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
# можно также сразу указывать, что у нас данные из словаря, orient='index' вместо transpose()
df = pd.DataFrame.from_dict(news, orient='index', columns = ['Ссылка','Время','Просмотры','Теги'])
df